In [3]:
# -*- coding: utf-8 -*-
"""
Pipeline này triển khai chiến lược xử lý dữ liệu thiếu mới, đã sửa lỗi
để đảm bảo tất cả các đặc trưng categorical được One-Hot Encode ở cuối.
"""

import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# =============================================================================
# 1. TẢI DỮ LIỆU GỐC
# =============================================================================
try:
    train_df_raw = pd.read_csv('train.csv')
    test_df_raw = pd.read_csv('test.csv')
    print("Đã tải dữ liệu thô thành công.")
except FileNotFoundError:
    print("Lỗi: Không tìm thấy file train.csv hoặc test.csv.")
    exit()

# =============================================================================
# 2. HÀM XỬ LÝ DỮ LIỆU (ĐÃ SỬA LỖI)
# =============================================================================
def apply_predictive_imputation(train_df, test_df):
    
    y_target = train_df['Survived']
    combined_df = pd.concat([train_df.drop('Survived', axis=1), test_df], ignore_index=True)
    print("Đã kết hợp train và test...")

    # --- Bước A: Xử lý các giá trị thiếu đơn giản ---
    combined_df['Embarked'].fillna(combined_df['Embarked'].mode()[0], inplace=True)
    combined_df['Fare'].fillna(combined_df['Fare'].median(), inplace=True)
    print("Đã điền các giá trị thiếu cho 'Embarked' và 'Fare'.")

    # --- Bước B: Feature Engineering sơ bộ ---
    combined_df['Title'] = combined_df['Name'].str.extract(' ([A-Za-z]+)\\.', expand=False)
    combined_df['Title'] = combined_df['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    combined_df['Title'] = combined_df['Title'].replace({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'})
    
    combined_df['FamilySize'] = combined_df['SibSp'] + combined_df['Parch'] + 1
    
    # --- Bước C: Dự đoán giá trị 'Age' bị thiếu ---
    print("Đang chuẩn bị để dự đoán 'Age'...")
    
    # Tạo một bản sao của DataFrame để thực hiện mã hóa tạm thời
    df_for_age_pred = combined_df.copy()
    
    # Mã hóa tạm thời các đặc trưng để đưa vào mô hình dự đoán Age
    df_for_age_pred['Sex'] = df_for_age_pred['Sex'].map({'male': 0, 'female': 1})
    df_for_age_pred['Embarked'] = df_for_age_pred['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    df_for_age_pred['Title'] = df_for_age_pred['Title'].map(title_mapping).fillna(0)
    
    age_features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Title', 'FamilySize']
    
    age_known = df_for_age_pred[df_for_age_pred['Age'].notna()]
    age_unknown = df_for_age_pred[df_for_age_pred['Age'].isna()]

    age_regressor = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
    age_regressor.fit(age_known[age_features], age_known['Age'])
    
    predicted_age = age_regressor.predict(age_unknown[age_features])
    
    # Điền giá trị đã dự đoán vào DataFrame gốc (combined_df)
    combined_df.loc[combined_df['Age'].isna(), 'Age'] = predicted_age
    print(f"Đã dự đoán và điền {len(predicted_age)} giá trị 'Age' bị thiếu.")

    # --- Bước D: Hoàn thiện Feature Engineering và Mã hóa cuối cùng ---
    combined_df['HasCabin'] = combined_df['Cabin'].notna().astype(int)
    combined_df['Deck'] = combined_df['Cabin'].fillna('U').str[0]
    
    # Xóa các cột không cần thiết
    combined_df.drop(columns=['Name', 'Ticket', 'Cabin', 'SibSp', 'Parch'], inplace=True)
    
    # *** SỬA LỖI Ở ĐÂY: ONE-HOT ENCODE TẤT CẢ CÁC CỘT CATEGORICAL CÒN LẠI ***
    # Xác định các cột categorical cần mã hóa
    categorical_cols = ['Sex', 'Embarked', 'Title', 'Deck']
    
    print(f"Đang thực hiện One-Hot Encoding cho các cột: {categorical_cols}")
    # Code đã sửa: Thêm dtype=int để đảm bảo kết quả là 1/0
    combined_df = pd.get_dummies(combined_df, columns=categorical_cols, drop_first=True, dtype=int)
    
    print("Đã hoàn thành xử lý và feature engineering.")

    # Tách lại thành train và test set
    train_final = combined_df.iloc[:len(train_df)]
    test_final = combined_df.iloc[len(train_df):]
    train_final['Survived'] = y_target

    return train_final, test_final

# =============================================================================
# 3. THỰC THI VÀ LƯU KẾT QUẢ
# =============================================================================
train_final_df, test_final_df = apply_predictive_imputation(train_df_raw, test_df_raw)

save_dir = "Data_clean_v2" 
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    print(f"Đã tạo thư mục '{save_dir}'")

try:
    X = train_final_df.drop(columns=['Survived'])
    y = train_final_df['Survived']
    X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    
    np.savez(f'{save_dir}/model_data.npz', X_train=X_train.values, X_validation=X_validation.values, y_train=y_train.values, y_validation=y_validation.values, columns=X.columns)
    train_final_df.to_csv(f'{save_dir}/train_cleaned.csv', index=False)
    test_final_df.to_csv(f'{save_dir}/test_cleaned.csv', index=False)
    print(f"\nĐã lưu tất cả các file vào thư mục '{save_dir}' thành công.")
    
    print("\nXem 5 dòng đầu của dữ liệu huấn luyện cuối cùng (đã One-Hot Encoded):")
    print(train_final_df.head())
    print("\nCác cột của dữ liệu cuối cùng:")
    print(train_final_df.columns.tolist())
    
except Exception as e:
    print(f"\nLỗi khi lưu file: {e}")

Đã tải dữ liệu thô thành công.
Đã kết hợp train và test...
Đã điền các giá trị thiếu cho 'Embarked' và 'Fare'.
Đang chuẩn bị để dự đoán 'Age'...


C:\Users\DELL\AppData\Local\Temp\ipykernel_17312\2173868225.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_df['Embarked'].fillna(combined_df['Embarked'].mode()[0], inplace=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_17312\2173868225.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alw

Đã dự đoán và điền 263 giá trị 'Age' bị thiếu.
Đang thực hiện One-Hot Encoding cho các cột: ['Sex', 'Embarked', 'Title', 'Deck']
Đã hoàn thành xử lý và feature engineering.

Đã lưu tất cả các file vào thư mục 'Data_clean_v2' thành công.

Xem 5 dòng đầu của dữ liệu huấn luyện cuối cùng (đã One-Hot Encoded):
   PassengerId  Pclass   Age     Fare  FamilySize  HasCabin  Sex_male  \
0            1       3  22.0   7.2500           2         0         1   
1            2       1  38.0  71.2833           2         1         0   
2            3       3  26.0   7.9250           1         0         0   
3            4       1  35.0  53.1000           2         1         0   
4            5       3  35.0   8.0500           1         0         1   

   Embarked_Q  Embarked_S  Title_Miss  ...  Title_Rare  Deck_B  Deck_C  \
0           0           1           0  ...           0       0       0   
1           0           0           0  ...           0       0       1   
2           0           1      

C:\Users\DELL\AppData\Local\Temp\ipykernel_17312\2173868225.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_final['Survived'] = y_target
